# Tutorial for Network Flow based Multi-Agent Path Planning

In this tutorial, the basic intuition and the reduction method of the network flow based MAPP algorithm will be introduced. For more detailed information, please refer to the original paper [1]. 

The network flow based MAPP algorithm consists of two steps. The first step is converting MAPP problem instance to the network flow problem instance. Then the second step is solving the converted problem with available solution methods for the network flow problem. In this material, we are going to consider only the first step, which is the innovation of this algorithm. Of course, the second step is also important step, but we will simply use an existing solver. 

First of all, it is necessary to understand the basic network flow problem. Note that there are several classes of network flow problem, but in this material, we will consider single commodity network flow problem. The following figure shows one of the example of the network flow problem. Network flow problem can be represented as directed graph, where the basic elements are nodes and directed edges. Some nodes have a certain amount of supplies and some other nodes have a certain amount of demands. Also, an edge has specific cost and capacity values. Then the question of the network flow problem is to find optimal assigment of the commodity, in terms of the cost, and in accordance with capacity, to meet all the supplies and demands. 

![title](../img/img1.jpg)

Then how the MAPP problem is related to network flow problem? To have some intuition about this, let's consider the following example. In the left, there is a grid world, and let's say this is a grid at time 0. Then in the right, there is the same grid world, which is for the time 1. And the agents (robots shown in the left grid) want to go to the nodes with charger signs (in the right grid) to charge their batteries. (Please note that, the grids on the left and the right figures are the same, but with different times.) Let also assume that robot can go to adjacent cell with unit time. Then now, can we encode this simple MAPP problem as network flow problem?

![title](../img/img2.jpg)

Naively, we can think about the below network flow problem. All the nodes in the time "t" world are connected to their adjacent nodes in the time "t+1" world. Then mark supply +1 to the origins and demand -1 to the goals. Also, assigning cost 1 and capacity 1 to each edge. If we solve this problem, supplies will be assigned to some edges, which can be interpreted as the "movement". Also, the optimaility can be interpreted as the minimum movement of the robots.

Of course this is not correct! (We shall fix some techinal problems step by step shortly.) However, this gives most of the idea of this algorithm. Now, with this intuition in mind, let's learn the exact way of converting MAPP problem to network flow problem.

![title](../img/img3.jpg)

For this purpose, let us consider the following example. The setting is same. Two robots want to go from their initial locations to charging stations. 

![title](../img/img4.jpg)

Now, let's build network flow problem step by step. In the following figures, there are maps for time step "t" and its subsequent time step "t+1". Also, let us consider the edge between nodes A and node B. (In fact, the same procedure will be applied for the other edges. So we only have to understand the procedure for a single edge.)

![title](../img/img5.jpg)

First of all, we want to model the movement as shown in the real world map in the following figure. As shown in the left, this can be modeled by adding the red directed edge.

![title](../img/img6.jpg)

Similarly, the green movement can be modeled by adding the green directed edge.

![title](../img/img7.jpg)

In addition, some robots may want to stay at the same node for the subsequent time steps. This can be modeled by adding parallel edges like blue edges shown in the following figure.

![title](../img/img8.jpg)

However, as you can expect, green and red edges above can incur collision between agents. So we have to fix this, as shown in the following figure. We added an artificial edge, shown as black edge, and assign capacity 1 to that edge. Then since only one robot can pass that edge, there will be no collision.

Now, let's assign cost and capacity to the edges carefully. Note that, the first value in the parenthesis is cost, and the second value is capacity. Since movement from node A to node B (or node B to node A, respectively) should have cost 1 (because it is single movement), only the black edge has to have a cost 1, and 0 cost for the others. And it is straightforward that stationary actions have cost 1 and capacity 1. However, one important note is, you should assign cost 0 to the stationary action, if the node is goal node. 

![title](../img/img9.jpg)

The last step is preventing the collision which can occur when an agent stays at some node and the other agent moves to that node from the other node. For example, in the above graph, if the agent 1 stays at node A for subsequent time steps, and the agent 2 moves from node B to node A, there will be collision.

This collision can be prevented by adding two more additional parallel edges as shown in the following figure. Note that, since these are artificial edges, the costs should be 0. 

![title](../img/img10.jpg)

Now we are done! The only thing that you should do is applying the above procedure for each edge for subsequent time steps, from t=0 to some fixed time horizon. Here, we will assume that some fixed time horizon is given, which is large enough to find the feasible solution. If you want to know how to find this horizon with more sophisticated way, please refer to the original paper.

![title](../img/img11.jpg)

Until now, we have learned how to reduce a MAPP problem to network flow problem. Now, let's think about what kind of MAPP problem can be correctly solved with this technique. In fact, not all of the MAPP problem can be solved with this technique. For example, what if the size of the robots is too big so that it cannot prevent a collision even though two robots are at the different nodes of the graph? Or, what if the angle between two edges is too small so that two robots travelling those two edges can collide each other? Therefore, to make it sufficient that this technique solve all the possible conflicts, we should have some assumption on the graph. We call this as collision-free unit-distance graph (CUG), and has the following properties:

1) Every edge has same length $D$.

2) Angle between two adjacent edges should be equal or greater than 90 degrees.

3) Size of the robot should be equal or less than $\sqrt{2}/4\cdot D$.

4) Every robot has the same speed.

[1] Yu, Jingjin, and Steven M. LaValle. "Multi-agent path planning and network flow." Algorithmic foundations of robotics X. Springer, Berlin, Heidelberg, 2013. 157-173.